# Process & Enrich Observables from Reports

## Setup

In [2]:
%%capture

import json
import csv
import vt
import nest_asyncio
import subprocess
import requests
from msticpy import init_notebook
from msticpy.nbtools.ti_browser import browse_results
import os
import jmespath
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()
init_notebook(namespace=globals());

VT_API = os.getenv('VT_API')
OSINT_LINKS = "./reports/reports_links.txt"
IOCPARSER = "https://api.iocparser.com/url"
PARSED_IOCS_DIR = "./reports/parsed_iocs_tc/"

## Parse IOCs online open-source reports

In [3]:
with open(OSINT_LINKS) as osint_links:
    types_to_parse = []
    for link in osint_links:
        request_data = {
            "url": link
        }

        headers = {
            'Content-Type': 'application/json'
        }
        iocparser_request = requests.post(IOCPARSER, headers=headers, json=request_data)
        parsed_iocs = iocparser_request.json()
        article_title = parsed_iocs['meta']['title']
        
        print("Article: ", article_title)
        print(parsed_iocs['meta']['url'])

        # Save IOCs to CSV
        csv_name = article_title.replace(" ", "_")
        
        with open(PARSED_IOCS_DIR + csv_name + ".csv", 'w') as csvfile:
            fieldnames = ['Type', 'Value', 'Rating', 'Confidence', 'Source', 'Description', 'Tags', 'Adversary', 'Document', 'Email', 'Incident', 'Signature', 'Threat']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()

            for attribute, values in parsed_iocs['data'].items():
                print()
                print(attribute)
                print("----------------------------------------")
                for value in values:
                    print(value)
                    if "FILE_HASH" in attribute:
                        writer.writerow({'Type': "File", 'Value': value})
                    elif attribute.startswith("IPv"):
                        writer.writerow({'Type': "Address", 'Value': value})
                    elif attribute == "EMAIL":
                        writer.writerow({'Type': "EmailAddress", 'Value': value})
                    elif attribute == "URL":
                        writer.writerow({'Type': "Url", 'Value': value})
                    elif attribute == "DOMAIN":
                        writer.writerow({'Type': "Host", 'Value': value})
                        

Article:  Hunter Becomes Hunted: Zebra2104 Hides a Herd of Malware
https://blogs.blackberry.com/en/2021/11/zebra2104

FILE_HASH_SHA1
----------------------------------------

DOMAIN
----------------------------------------
trashborting.com
kavamennci.us
bertolinnj.us
eixirienhj.us
infuuslx.us
protonmail.com
mail.premiumclube.org.br
auswalzenna.us
megafonasgc.us
magesty.in-expedition.com
mentiononecommon.com
trashborting.com
ns1.entrydns.net
ns3.entrydns.net
ns4.entrydns.net
mipancepezc.us
supercombinating.com
ticket-one-two.com
ns2.entrydns.net
any.run
ticket-one-two.com
lionarivv.us
whois.namecheap.com
zensingergy.us
booking-sales.com
xperi.link
publicdomainregistry.com
okergeeliw.us

IPv6
----------------------------------------

YARA_RULE
----------------------------------------

IPv4
----------------------------------------
87.120.37.120
91.92.109.174

EMAIL
----------------------------------------

FILE_NAME
----------------------------------------
SecurityHost.exe
zpsxxla.php
zxl

## Query IOCs within OSINT datasets

In [7]:
ti_lookup = TILookup(providers=["OTX", "VirusTotal"])

iocs_msticpy = []

for report_csv in os.listdir(PARSED_IOCS_DIR):
    if(report_csv.endswith('.csv')):
        with open(PARSED_IOCS_DIR + report_csv) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                iocs_msticpy.append(row['Value'])

results = ti_lookup.lookup_iocs(data=iocs_msticpy, providers=["OTX", "VirusTotal"])

## Tabular View of Results

In [8]:
results_columns_filtered = results[["Ioc", "Provider", "Details", "Severity"]]
results_columns_filtered.sort_values("Ioc")

,Ioc,Provider,Details,Severity
20,87.120.37.120,VirusTotal,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 2, 'detected_urls': ['...",high
20,87.120.37.120,OTX,"{'pulse_count': 34, 'names': ['Malware Command and Control IPs', 'StrongPity, Zebra2104, Phobos,...",high
23,8844d234d9e18e29f01ff8f64db70274c02953276a2cd1a1a05d07e7e1feb55c,VirusTotal,"{'verbose_msg': 'Scan finished, information embedded', 'response_code': 1, 'positives': 59, 'res...",high
23,8844d234d9e18e29f01ff8f64db70274c02953276a2cd1a1a05d07e7e1feb55c,OTX,"{'pulse_count': 2, 'names': ['StrongPity, Zebra2104, Phobos, MountLocker IoCs', 'Hunter Becomes ...",high
21,91.92.109.174,VirusTotal,"{'verbose_msg': 'IP address in dataset', 'response_code': 1, 'positives': 1, 'detected_urls': ['...",warning
21,91.92.109.174,OTX,"{'pulse_count': 1, 'names': ['Hunter Becomes Hunted: Zebra2104 Hides a Herd of Malware'], 'tags'...",warning
6,auswalzenna.us,OTX,"{'pulse_count': 7, 'names': ['Email Malspam Campaign Infrastructure Analysis', 'StrongPity, Zebr...",high
6,auswalzenna.us,VirusTotal,"{'verbose_msg': 'Domain found in dataset', 'response_code': 1, 'positives': 0, 'detected_urls': ...",information
2,bertolinnj.us,OTX,"{'pulse_count': 7, 'names': ['Email Malspam Campaign Infrastructure Analysis', 'StrongPity, Zebr...",high
2,bertolinnj.us,VirusTotal,"{'verbose_msg': 'Domain found in dataset', 'response_code': 1, 'positives': 3, 'detected_urls': ...",high


## Generate IOC Selector Panel

In [6]:
ti_selector = browse_results(data=results, height="300px", width="800px")
ti_selector

## [VirusTotal] Get Triggered Yara and IDS Rules

In [3]:
import vt

vt_client = vt.Client(VT_API)
vt_file_db = []
for report_csv in os.listdir(PARSED_IOCS_DIR):
    if report_csv.endswith('.csv'):
        with open(PARSED_IOCS_DIR + report_csv) as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                if(row['Type'] == "File"):
                    file_hash = row['Value']
                    try:
                        vt_file = vt_client.get_object("/files/" + file_hash)
                        vt_file_db.append(vt_file)
                    except:
                        pass
vt_client.close()

In [4]:
for vt_file in vt_file_db:
    yara_list = jmespath.search('crowdsourced_yara_results[*].rule_name', vt_file)
    ids_list = jmespath.search('crowdsourced_ids_results[*].rule_msg', vt_file)
    if (yara_list is not None or ids_list is not None):
        print("\n=============================\nRules for " + file_hash + "\n=============================")                    
        if(yara_list is not None and len(yara_list) > 0):
            print("Yara:")
            print(yara_list)
            print()
        if(ids_list is not None and len(ids_list) > 0):
            print('IDS:')
            print(ids_list)
    if "popular_threat_classification" in vt_file.to_dict():
        print("\nThreat Classification:", vt_file.popular_threat_classification['suggested_threat_label'])